In [54]:
import tiktoken

In [55]:
tokenizer=tiktoken.get_encoding("gpt2")


In [56]:
with open("the-verdict.txt","r") as f:
    raw_text=f.read()

encoded_text=tokenizer.encode(raw_text)
print(len(encoded_text))    



5145


In [57]:
context_size=4

x=encoded_text[:context_size]
y=encoded_text[1:context_size+1]
   
print(x)
print("   ",y)   


[40, 367, 2885, 1464]
    [367, 2885, 1464, 1807]


In [58]:
for i in range(1,context_size+1):
    context=encoded_text[:i]
    desired=encoded_text[i]
    print(context,"===>",desired)

[40] ===> 367
[40, 367] ===> 2885
[40, 367, 2885] ===> 1464
[40, 367, 2885, 1464] ===> 1807


In [59]:
for i in range(1,context_size+1):
    context = encoded_text[:i]  
    desired = encoded_text[i]
    print(tokenizer.decode(context), "===>", tokenizer.decode([desired]))


I ===>  H
I H ===> AD
I HAD ===>  always
I HAD always ===>  thought


Datasets and Data Loader


Datasets and Dataloader allows us to load data in more effificient manner

we will implement Data Loader that Fatches the Data Using sliding window approach   

In [60]:
from torch.utils.data import Dataset, DataLoader
import torch

In [61]:
class GPTversion1(Dataset):
    def __init__(self,text#text file to be read
                 ,tokenizer #byte pair tokenizer
                 ,max_length #cotext size
                 ,stride):
        self.input_ids=[]
        self.target_ids=[]
        
        token_ids=tokenizer.encode(text,allowed_special={'<|endoftext|>'})

        for i in range(0,len(token_ids)-max_length,stride):
            input_chunk=token_ids[i:i+max_length]
            target_chunk=token_ids[i+1:i+max_length+1]
            self.input_ids.append(torch.tensor(input_chunk))
            self.target_ids.append(torch.tensor(target_chunk))


    def __len__(self):
        return len(self.input_ids)
    
    def __getitem__(self,i):
        return self.input_ids[i],self.target_ids[i]

In [62]:
#create Dataset loader
def create_dataset_Loader_V1(text,batch_size=4#number of CPU cores to use
                         ,max_length=256,
                         stride=128,
                         shuffle=True,
                         drop_last=True,
                         num_workers=0#number of CPU threads that we can use simultaneously
                         ):
    #initialize the tokenizer
    tokenizer=tiktoken.get_encoding("gpt2")
    dataset=GPTversion1(text,tokenizer,max_length,stride)
    dataLoader=DataLoader(dataset,batch_size=batch_size,shuffle=shuffle,drop_last=drop_last,num_workers=num_workers)
    return dataLoader





In [66]:
import torch
with open("the-verdict.txt","r") as f:
    raw_text=f.read()
dataloader=create_dataset_Loader_V1(raw_text,batch_size=1,max_length=4,
                                    stride=1)
data_iter=iter(dataloader)
first_batch=next(data_iter)
print(first_batch)




[tensor([[6653, 6563, 2951, 6348]]), tensor([[6563, 2951, 6348, 5391]])]
